# code to run llava using Selenium for multiple images

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import math
import ast
import json
import re
pattern = r'\[\d{2}:\d{2}:\d{2}\]'

c:\Users\iamma\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
os.listdir()

['act_dialogue_dict_organized_final.json',
 'act_dialogue_dict_organized_final_with_image_urls.json',
 'chromedriver_win32',
 'Creation of D1 template dataset.ipynb',
 'D1_preprocessing_code_and_data',
 'Getting_all_image_urls.ipynb',
 'Get_images_from_the_zip_files.ipynb',
 'images_final_required',
 'image_descriptions_all_final.csv',
 'image_urls.pkl',
 'meetup-data',
 'meetup_d1_format_dataset_original_dialogs.csv',
 'meetup_d1_format_dataset_original_dialogs_zipped.zip',
 'Running Llava using Selenium.ipynb',
 'to_do_images.pkl']

In [3]:
files_path = "D:\inria_work_dataset\meetup-data\original_dialogs\\"

In [4]:
image_df = pd.read_csv("D:\inria_work_dataset\image_descriptions_all_final.csv")
#convert the images in image_path and description in description to a dict
image_descriptions_dict = {}
for i in range(len(image_df)):
    image_descriptions_dict[image_df.iloc[i]['image_path']] = image_df.iloc[i]['description']

In [5]:
files = os.listdir(files_path)
len(files), files[:2]

(430, ['dial_0.csv', 'dial_1.csv'])

In [6]:
df = pd.read_csv(files_path + "dial_102.csv")
df.head(20)

,Unnamed: 0,time,public,A-private,B-private,user,receiver
0,0,0 days 00:00:00,A:,A:,NaN,A,All
1,1,0 days 00:00:00,B:,NaN,B:,B,All
2,2,0 days 00:00:00,GM:,GM:,GM:,GM,All
3,3,0 days 00:00:01,NaN,GM: cmd: listen_to,GM: cmd: listen_to,GM,All
4,4,0 days 00:00:01,NaN,GM: cmd: listen_to,GM: cmd: listen_to,GM,All
5,5,0 days 00:00:01,NaN,GM: cmd: listen_to,GM: cmd: listen_to,GM,All
6,6,0 days 00:00:01,NaN,GM: cmd: listen_to,GM: cmd: listen_to,GM,All
7,7,0 days 00:00:01,NaN,GM: cmd: listen_to,GM: cmd: listen_to,GM,All
8,8,0 days 00:00:01,NaN,GM: cmd: listen_to,GM: cmd: listen_to,GM,All
9,9,0 days 00:00:02,NaN,GM: cmd: listen_to,GM: cmd: listen_to,GM,All


In [7]:
df['time'][0][7:]

'00:00:00'

In [8]:
df['public'][0]

'A: '

In [9]:
def give_input_outputs(df):
    time_stamp = []
    convo_flow = []
    a_image_description = []
    b_image_description = []

    for convo,a_image, b_image, time in zip(df['public'], df['A-private'], df['B-private'], df['time']):
        if type(convo) == str:
            convo_flow.append(convo)
            time_stamp.append(time[7:])
        else:
            convo_flow.append("")
            time_stamp.append(time[7:])
            
        if ((type(a_image)== str ) and ("url" in a_image)):
            name = a_image.replace(" ", "")
            name = name.split("/")[-1]
            a_image_description.append(image_descriptions_dict[name])
        else:
            a_image_description.append("")

        if ((type(b_image)== str ) and ("url" in b_image)):
            name = b_image.replace(" ", "")
            name = name.split("/")[-1]
            b_image_description.append(image_descriptions_dict[name])
        else:
            b_image_description.append("")
    
    return convo_flow,a_image_description, b_image_description, time_stamp

convo,a_image_description, b_image_description, time_stamp = give_input_outputs(df)


In [10]:
    
def create_dataset(convo,a_image_description, b_image_description, time_stamp):
    prior = " Instructions : Here is a conversation between two Participants A and B who are in different rooms. Each room has a type.  Participants are in one room at a time. They communicate with each other and describe their rooms. The participants can move to different rooms and describe their new room to the other participant until they reach the same room. The aim is to meet in the same room. The descriptions of the rooms that the participants currently saw are provided in text in between the utterances. The current room description of User A starts with a token <Image A> and the current room description of User B starts with a token <Image B>. Every utterance from A or B is preceded with a timestamp closed under brackets. GM is the third person who provides essential information regarding the game to both the participants."

    flags = [1]*len(convo)
    inputs = []
    outputs = []
    current_convo = prior
    for i in range(len(convo)-1):
        if a_image_description[i] != "":
            current_convo += " <Image A> " + a_image_description[i]
            inputs.append(current_convo)
            outputs.append(convo[i+1])
        if b_image_description[i] != "":
            current_convo += " <Image B> " + b_image_description[i]
            inputs.append(current_convo)
            outputs.append(convo[i+1])

        #if convo[i] != "":
        if convo[i] != "A: " and (convo[i] != "B: ") and (convo[i] != "GM: ") and (".w" not in convo[i]) and (".e" not in convo[i]) and (".n" not in convo[i]) and (".s" not in convo[i]):
            if convo[i][:2] == "A:":
                current_convo += (" ["+ time_stamp[i] + '] ')*flags[i] + convo[i] + (" ["+ time_stamp[i+1] + '] ')*flags[i+1]
                flags[i] = 0
                flags[i+1] = 0
            elif convo[i][:2] == "B:":
                current_convo += (" ["+ time_stamp[i] + '] ')*flags[i] + convo[i] + (" ["+ time_stamp[i+1] + '] ')*flags[i+1]
                flags[i] = 0
                flags[i+1] = 0
            else:    
                current_convo += " "  + convo[i]
                
            inputs.append(current_convo)
            outputs.append(convo[i+1])

        # now remove those inputs, whose outputs are empty or "A:" or "B:" or "GM:" and also remove the the ones that have GM in output
        inputs = [i for i,o in zip(inputs,outputs) if ((o != "") and (o != "A: ") and (o != "B: ") and (o != "GM: ") and ("GM:" not in o))]
        outputs = [o for o in outputs if ((o != "") and (o != "A: ") and (o != "B: ") and (o != "GM:") and ("GM:" not in o))]
        # remove \n from inputs and outputs
        inputs = [i.replace("\n", "") for i in inputs]
        outputs = [o.replace("\n", "") for o in outputs]
        
    return inputs, outputs



In [12]:
flow = pd.DataFrame({"inputs":inputs, "outputs":outputs})
flow['inputs'] = flow['inputs'].apply(lambda x: x.replace("\n", " "))
flow.head(20)

NameError: name 'inputs' is not defined

In [131]:
# test the functions for dial_102.csv
df = pd.read_csv(files_path + "dial_102.csv")
convo,a_image_description, b_image_description, time_stamp = give_input_outputs(df)
inputs, outputs = create_dataset(convo,a_image_description, b_image_description, time_stamp)


In [13]:
big_input_list, big_output_list = [], []
for file in tqdm(files):
    df = pd.read_csv(files_path + file)
    convo,a_image_description, b_image_description, time_stamp = give_input_outputs(df)
    inputs, outputs = create_dataset(convo,a_image_description, b_image_description, time_stamp)
    big_input_list.extend(inputs)
    big_output_list.extend(outputs)

  0%|          | 0/430 [00:00<?, ?it/s]

100%|██████████| 430/430 [00:04<00:00, 98.88it/s] 


In [14]:
len(big_input_list), len(big_output_list)

(5690, 5690)

In [15]:
df_d1 = pd.DataFrame({"inputs":big_input_list, "outputs":big_output_list})
df_d1.head(20)

,inputs,outputs
0,Instructions : Here is a conversation between...,B: i'm in the playroom
1,Instructions : Here is a conversation between...,B: you have to go west
2,Instructions : Here is a conversation between...,A: What does it look like
3,Instructions : Here is a conversation between...,A: describe it please
4,Instructions : Here is a conversation between...,"A: I'm in one as well, I see a small chalkboa..."
5,Instructions : Here is a conversation between...,A: Please describe what you see in your playroom
6,Instructions : Here is a conversation between...,A: Hello????????
7,Instructions : Here is a conversation between...,B: has a little boy
8,Instructions : Here is a conversation between...,B: with a table
9,Instructions : Here is a conversation between...,B: and blyue chair


In [16]:
"""here are some utterances I see - 
w/
'/n
.w
 e/
.e
.n
[/n]orth
.s

/e 
n/
s/
we need to remove them and replace them with ""

"""
def clean_string(text):
    text = text.replace("/n", "")
    text = text.replace(".w", "")
    text = text.replace(".e", "")
    text = text.replace(".n", "")
    text = text.replace(".s", "")
    text = text.replace("w/", "")
    text = text.replace("e/", "")
    text = text.replace("n/", "")
    text = text.replace("s/", "")
    text = text.replace("[/n]", "")
    text = text.replace("  ", " ")
    return text

df_d1['inputs'] = df_d1['inputs'].apply(lambda x: clean_string(x))
df_d1['outputs'] = df_d1['outputs'].apply(lambda x: clean_string(x))
df_d1.head(20)

,inputs,outputs
0,Instructions : Here is a conversation between...,B: i'm in the playroom
1,Instructions : Here is a conversation between...,B: you have to go west
2,Instructions : Here is a conversation between...,A: What does it look like
3,Instructions : Here is a conversation between...,A: describe it please
4,Instructions : Here is a conversation between...,"A: I'm in one as well, I see a small chalkboar..."
5,Instructions : Here is a conversation between...,A: Please describe what you see in your playroom
6,Instructions : Here is a conversation between...,A: Hello????????
7,Instructions : Here is a conversation between...,B: has a little boy
8,Instructions : Here is a conversation between...,B: with a table
9,Instructions : Here is a conversation between...,B: and blyue chair


In [17]:
df_d1.to_csv("meetup_d1_format_dataset_original_dialogs.csv", index=False)